Lab 2 --> Implementing a Multi-Class Classification Model using Deep neural network

---------

Step 1:
* Import all the neccesary Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import h5py
from PIL import Image
from scipy import ndimage
from sklearn.model_selection import train_test_split

%matplotlib inline

* Drive mount

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Give access to the Files Te.h5 and Tr.h5

In [4]:
def inspect_h5_file(file_name):
  with h5py.File(file_name,'r') as file:
    def print_structure(name, obj):
      print(name)
    file.visititems(print_structure)

inspect_h5_file('/content/drive/MyDrive/Deep Learning Lab Activities/Lab_2/Te (2).h5')
inspect_h5_file('/content/drive/MyDrive/Deep Learning Lab Activities/Lab_2/Tr (2).h5')

images
labels
images
labels


Step 2:
* Load the Dataset

In [5]:
def load_dataset():
    train_dataset = h5py.File('/content/drive/MyDrive/Deep Learning Lab Activities/Lab_2/Tr (2).h5', "r")
    train_set_x_orig = np.array(train_dataset["images"][:])
    train_set_y_orig = np.array(train_dataset["labels"][:])

    test_dataset = h5py.File('/content/drive/MyDrive/Deep Learning Lab Activities/Lab_2/Te (2).h5', "r")
    test_set_x_orig = np.array(test_dataset["images"][:])
    test_set_y_orig = np.array(test_dataset["labels"][:])

    classes = np.unique(train_set_y_orig)

    train_set_y_orig = train_set_y_orig.reshape((1, train_set_y_orig.shape[0]))
    test_set_y_orig = test_set_y_orig.reshape((1, test_set_y_orig.shape[0]))

    return train_set_x_orig, train_set_y_orig, test_set_x_orig, test_set_y_orig, classes

# Load and preprocess data
train_x_orig, train_y_orig, test_x_orig, test_y_orig, classes = load_dataset()

Step 3:   Preprocess the Dataset

In [6]:
def preprocess_data(train_x_orig, test_x_orig):
    # Flatten and normalize
    train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1) / 255.0
    test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1) / 255.0

    return train_x_flatten.T, test_x_flatten.T

# Preprocess data
train_x, test_x = preprocess_data(train_x_orig, test_x_orig)


Step 2: Convert Labels to One-Hot Encoding

In [7]:
def one_hot_encode(labels, num_classes):
    return np.eye(num_classes)[labels.reshape(-1)].T

# Convert labels to one-hot encoding
train_y = one_hot_encode(train_y_orig, len(classes))
test_y = one_hot_encode(test_y_orig, len(classes))


Step 3: Initialize Parameters

In [8]:
def initialize_parameters(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))

    return parameters


Step 4: Implement Forward Propagation

In [9]:
def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def relu(Z):
    return np.maximum(0, Z)

def forward_propagation(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2

    for l in range(1, L):
        A_prev = A
        Z = np.dot(parameters['W' + str(l)], A_prev) + parameters['b' + str(l)]
        A = relu(Z)
        caches.append((A_prev, Z))

    ZL = np.dot(parameters['W' + str(L)], A) + parameters['b' + str(L)]
    AL = softmax(ZL)
    caches.append((A, ZL))

    return AL, caches


Step 5: Implement Backward Propagation

In [10]:
def backward_propagation(AL, Y, caches, parameters):
    grads = {}
    L = len(caches)
    m = AL.shape[1]

    dZL = AL - Y
    grads['dW' + str(L)] = np.dot(dZL, caches[L-1][0].T) / m
    grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m

    for l in reversed(range(1, L)):
        dA = np.dot(parameters['W' + str(l+1)].T, dZL)
        dZ = dA * (caches[l-1][1] > 0)  # ReLU derivative
        grads['dW' + str(l)] = np.dot(dZ, caches[l-1][0].T) / m
        grads['db' + str(l)] = np.sum(dZ, axis=1, keepdims=True) / m
        dZL = dZ

    return grads


Step 6: Update Parameters Using Gradient Descent

In [11]:
def update_parameters(parameters, grads, learning_rate):
    L = len(parameters) // 2

    for l in range(1, L+1):
        parameters['W' + str(l)] -= learning_rate * grads['dW' + str(l)]
        parameters['b' + str(l)] -= learning_rate * grads['db' + str(l)]

    return parameters


Step 7: Train the Model

In [12]:
def model(X_train, Y_train, layers_dims, learning_rate=0.0075, num_iterations=3000):
    parameters = initialize_parameters(layers_dims)

    for i in range(num_iterations):
        AL, caches = forward_propagation(X_train, parameters)
        cost = -np.sum(Y_train * np.log(AL)) / Y_train.shape[1]
        grads = backward_propagation(AL, Y_train, caches, parameters)
        parameters = update_parameters(parameters, grads, learning_rate)

        if i % 100 == 0:
            print(f"Cost after iteration {i}: {cost}")

    return parameters


Step 8: Make Predictions and Evaluate the Model

In [13]:
def predict(X, parameters):
    AL, _ = forward_propagation(X, parameters)
    return np.argmax(AL, axis=0)

def accuracy(predictions, Y):
    return np.mean(predictions == np.argmax(Y, axis=0)) * 100

# Define the layers dimensions
layers_dims = [train_x.shape[0], 64, 32, len(classes)]  # Example: 3 layers

# Train the model
parameters = model(train_x, train_y, layers_dims)

# Test the model
train_predictions = predict(train_x, parameters)
test_predictions = predict(test_x, parameters)

print(f"Train Accuracy: {accuracy(train_predictions, train_y)}%")
print(f"Test Accuracy: {accuracy(test_predictions, test_y)}%")


Cost after iteration 0: 1.6095547093839042
Cost after iteration 100: 1.6092674945966423
Cost after iteration 200: 1.6089591991813577
Cost after iteration 300: 1.6085154638854235
Cost after iteration 400: 1.6077827417891644
Cost after iteration 500: 1.6063895043725838
Cost after iteration 600: 1.6032586298706992
Cost after iteration 700: 1.594881503122156
Cost after iteration 800: 1.5714319004422415
Cost after iteration 900: 1.5296963221996325
Cost after iteration 1000: 1.4732951578677849
Cost after iteration 1100: 1.3655926718972746
Cost after iteration 1200: 1.1706488018320444
Cost after iteration 1300: 1.0078406534158504
Cost after iteration 1400: 0.8277944358635745
Cost after iteration 1500: 0.6769252046990993
Cost after iteration 1600: 0.7112029893428061
Cost after iteration 1700: 0.3569321067698971
Cost after iteration 1800: 0.5169079344750432
Cost after iteration 1900: 0.22884127862918036
Cost after iteration 2000: 0.19092676423691204
Cost after iteration 2100: 0.1337464687477770